In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data/raw.csv', encoding = "ISO-8859-1", dtype={'CustomerID': str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


In [3]:
def preprocessing(df: pd.DataFrame):
  df = df.copy()

  df.dropna(inplace=True, subset=['CustomerID', 'Description'])
  df.drop_duplicates(inplace=True)

  # drop cancelled invoice in InvoiceNo
  df = df[~df['InvoiceNo'].str.startswith('C')]

  # format datetime type
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

  return df

In [4]:
def revenue_by_day(df: pd.DataFrame):
    df = df.copy()

    # group by Date, calculate sum of total price and keep time features such as weekday, month, ...
    df = df.groupby(['Date', 'Year', 'Quarter', 'Month', 'Weekday', 'DayOfYear']).agg({'Revenue': 'sum'}).reset_index()

    return df

def feature_engineer(df: pd.DataFrame):
    df = df.copy()

    df['Revenue'] = df['Quantity'] * df['UnitPrice']
    df['Date'] = df['InvoiceDate'].dt.date
    df['Year'] = df['InvoiceDate'].dt.year
    df['Quarter'] = df['InvoiceDate'].dt.quarter
    df['Month'] = df['InvoiceDate'].dt.month
    df['Weekday'] = df['InvoiceDate'].dt.weekday
    df['DayOfYear'] = df['InvoiceDate'].dt.dayofyear

    # drop InvoiceDate
    df.drop(columns=['InvoiceDate'], inplace=True)

    # filter
    lower_bound = df['Revenue'].quantile(0.01)
    upper_bound = df['Revenue'].quantile(0.99)
    df = df[(df['Revenue'] >= lower_bound) & (df['Revenue'] <= upper_bound)]

    # group
    df = revenue_by_day(df)

    return df

In [5]:
clean_df = preprocessing(df)
clean_df = feature_engineer(clean_df)
clean_df.head()

,Date,Year,Quarter,Month,Weekday,DayOfYear,Revenue
0,2010-12-01,2010,4,12,2,335,35376.36
1,2010-12-02,2010,4,12,3,336,35914.33
2,2010-12-03,2010,4,12,4,337,19312.81
3,2010-12-05,2010,4,12,6,339,28513.18
4,2010-12-06,2010,4,12,0,340,26574.17


In [6]:
clean_df.describe()

,Year,Quarter,Month,Weekday,DayOfYear,Revenue
count,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000
mean,2010.934426,2.534426,6.636066,2.678689,185.770492,22036.169213
std,0.247942,1.126663,3.465337,1.947089,105.030414,9794.438586
min,2010.000000,1.000000,1.000000,0.000000,4.000000,3439.100000
25%,2011.000000,2.000000,4.000000,1.000000,93.000000,15742.850000
50%,2011.000000,3.000000,7.000000,3.000000,188.000000,20024.800000
75%,2011.000000,4.000000,10.000000,4.000000,278.000000,27584.300000
max,2011.000000,4.000000,12.000000,6.000000,357.000000,57240.970000


In [7]:
# tìm ngày gần nhất và xa nhất
print(clean_df['Date'].min())
print(clean_df['Date'].max())

# tính số ngày trong khoảng đó
print((clean_df['Date'].max() - clean_df['Date'].min()).days)

2010-12-01
2011-12-09
373


In [8]:
# tạo dữ liệu những ngày thiếu
clean_df = clean_df.set_index('Date')
clean_df = clean_df.asfreq('D')
clean_df = clean_df.reset_index()
clean_df.head()

,Date,Year,Quarter,Month,Weekday,DayOfYear,Revenue
0,2010-12-01,2010.0,4.0,12.0,2.0,335.0,35376.36
1,2010-12-02,2010.0,4.0,12.0,3.0,336.0,35914.33
2,2010-12-03,2010.0,4.0,12.0,4.0,337.0,19312.81
3,2010-12-04,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-12-05,2010.0,4.0,12.0,6.0,339.0,28513.18


In [10]:
def rebuild_time_features(df):
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    df['Quarter'] = df['Date'].dt.quarter
    df['Month'] = df['Date'].dt.month
    df['Weekday'] = df['Date'].dt.weekday
    df['DayOfYear'] = df['Date'].dt.dayofyear
    return df


clean_df = rebuild_time_features(clean_df)
clean_df['Revenue'] = clean_df['Revenue'].fillna(clean_df['Revenue'].interpolate())
clean_df.head()

,Date,Year,Quarter,Month,Weekday,DayOfYear,Revenue
0,2010-12-01,2010,4,12,2,335,35376.360
1,2010-12-02,2010,4,12,3,336,35914.330
2,2010-12-03,2010,4,12,4,337,19312.810
3,2010-12-04,2010,4,12,5,338,23912.995
4,2010-12-05,2010,4,12,6,339,28513.180


In [11]:
clean_df.to_csv('data/clean_by_day.csv', index=False)